In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

pd.set_option("display.max_columns", 50)
pd.set_option("display.width", 140)

In [2]:
CSV_PATH = "avax_data.csv"  # <-- cambia esto

df_raw = pd.read_csv(CSV_PATH)
df_raw.head(10)

,datetime,symbol,open,high,low,close,volume
0,2022-01-23 01:00:00,Binance:AVAXUSD,59.853796,66.889643,59.038229,64.878152,268105.18
1,2022-01-24 01:00:00,Binance:AVAXUSD,64.852785,64.854387,55.236679,63.088485,224702.52
2,2022-01-25 01:00:00,Binance:AVAXUSD,62.978926,70.658855,61.710777,68.027966,206129.22
3,2022-01-26 01:00:00,Binance:AVAXUSD,68.057846,73.243396,64.502906,65.886976,250217.52
4,2022-01-27 01:00:00,Binance:AVAXUSD,65.879000,67.951789,61.139137,64.792479,144499.26
5,2022-01-28 01:00:00,Binance:AVAXUSD,64.727717,67.496031,61.775062,67.092026,112943.08
6,2022-01-29 01:00:00,Binance:AVAXUSD,67.060714,73.279454,66.894105,71.950964,191216.49
7,2022-01-30 01:00:00,Binance:AVAXUSD,71.952324,72.399777,66.854233,68.080713,112641.92
8,2022-01-31 01:00:00,Binance:AVAXUSD,68.013545,70.703146,64.298606,69.951072,156613.82
9,2022-02-01 01:00:00,Binance:AVAXUSD,69.949433,73.919883,68.463116,73.350603,196015.42


In [3]:
df = df_raw.copy()

# Normaliza nombres de columnas por si vienen con espacios
df.columns = [c.strip() for c in df.columns]

required_cols = ["datetime", "symbol", "open", "high", "low", "close", "volume"]
missing_required = [c for c in required_cols if c not in df.columns]
if missing_required:
    raise ValueError(f"Faltan columnas obligatorias: {missing_required}")

# Parse datetime
df["datetime"] = pd.to_datetime(df["datetime"], errors="coerce")

# Convierte OHLCV a numérico (si hay strings raros, quedarán NaN)
for c in ["open", "high", "low", "close", "volume"]:
    df[c] = pd.to_numeric(df[c], errors="coerce")

# Ordena y quita duplicados exactos
df = df.sort_values(["symbol", "datetime"]).reset_index(drop=True)

df.head()

,datetime,symbol,open,high,low,close,volume
0,2022-01-23 01:00:00,Binance:AVAXUSD,59.853796,66.889643,59.038229,64.878152,268105.18
1,2022-01-24 01:00:00,Binance:AVAXUSD,64.852785,64.854387,55.236679,63.088485,224702.52
2,2022-01-25 01:00:00,Binance:AVAXUSD,62.978926,70.658855,61.710777,68.027966,206129.22
3,2022-01-26 01:00:00,Binance:AVAXUSD,68.057846,73.243396,64.502906,65.886976,250217.52
4,2022-01-27 01:00:00,Binance:AVAXUSD,65.879000,67.951789,61.139137,64.792479,144499.26


### Chequeos generales: nulos, faltantes, duplicados, rangos

In [4]:
def general_quality_report(df: pd.DataFrame) -> pd.DataFrame:
    report = []

    # Nulos por columna
    nulls = df.isna().sum()
    null_pct = (nulls / len(df) * 100).round(3)

    for col in df.columns:
        report.append({
            "column": col,
            "null_count": int(nulls[col]),
            "null_pct": float(null_pct[col]),
            "dtype": str(df[col].dtype),
        })

    rep = pd.DataFrame(report).sort_values(["null_count", "column"], ascending=[False, True])

    # Duplicados por (symbol, datetime)
    dup_key = df.duplicated(subset=["symbol", "datetime"]).sum()
    print(f"Filas: {len(df):,}")
    print(f"Duplicados por clave (symbol, datetime): {dup_key:,}")

    # Fechas inválidas
    invalid_dt = df["datetime"].isna().sum()
    print(f"Datetimes inválidos (NaT): {invalid_dt:,}")

    # Sanity OHLC
    if all(c in df.columns for c in ["open","high","low","close"]):
        bad_hl = (df["high"] < df["low"]).sum()
        out_close = ((df["close"] < df["low"]) | (df["close"] > df["high"])).sum()
        print(f"high < low: {bad_hl:,}")
        print(f"close fuera de [low, high]: {out_close:,}")

    return rep

quality_df = general_quality_report(df)
quality_df

Filas: 1,461
Duplicados por clave (symbol, datetime): 0
Datetimes inválidos (NaT): 0
high < low: 0
close fuera de [low, high]: 0


,column,null_count,null_pct,dtype
5,close,0,0.0,float64
0,datetime,0,0.0,datetime64[us]
3,high,0,0.0,float64
4,low,0,0.0,float64
2,open,0,0.0,float64
1,symbol,0,0.0,str
6,volume,0,0.0,float64


### Estadísticas

In [5]:
stats = df.groupby("symbol")[["open","high","low","close","volume"]].describe().round(6)
stats

open                                                                               high                        \
                  count       mean        std       min        25%       50%        75%        max   count       mean        std   
symbol                                                                                                                             
Binance:AVAXUSD  1461.0  27.306465  17.400839  8.769508  16.481796  21.95125  33.175058  97.957614  1461.0  28.387681  18.264185   

                                                                           low                                                        \
                      min        25%        50%        75%         max   count       mean        std       min        25%        50%   
symbol                                                                                                                                 
Binance:AVAXUSD  8.917922  17.124063  22.794906  34.277915  103.646711  1461.0  26.181692  16.582783  8.639302  15.657523  21.056082   

                                       close                                                                             volume  \
                      75%        max   count      mean        std       min        25%        50%        75%        max   count   
symbol                                                                                                                            
Binance:AVAXUSD  31.64962  95.765336  1461.0  27.27251  17.384943  8.779517  16.355528  21.919428  33.143416  98.030571  1461.0   

                                                                                                    
                          mean            std       min       25%        50%       75%         max  
symbol                                                                                              
Binance:AVAXUSD  148437.601075  147288.926016  15162.88  65056.59  107980.47  174220.3  1879334.56

### Verificar que está en un intervalo de 4 años seguidos

In [6]:
from dateutil.relativedelta import relativedelta

def check_four_year_window(df_symbol: pd.DataFrame):
    s = df_symbol.dropna(subset=["datetime"]).sort_values("datetime")
    if s.empty:
        return {"ok": False, "reason": "SIN_FECHAS_VALIDAS"}

    start = s["datetime"].min()
    end = s["datetime"].max()

    # A) Diferencia exacta en meses (48 meses = 4 años)
    # Tomamos el "start" y sumamos 4 años: start + 4 años
    target_end = start + relativedelta(years=4)

    # B) Diferencia en días (aprox, útil para ver si "se pasa" o "no llega")
    delta_days = (end - start).total_seconds() / 86400.0

    # Criterio estricto: end debe estar en [target_end - 2d, target_end + 2d] (ajústalo si quieres)
    # Esto ayuda con series horarias/diarias que no terminan exactamente a misma hora.
    tolerance_days = 2
    strict_ok = abs((end - target_end).total_seconds()) <= tolerance_days * 86400

    return {
        "symbol": s["symbol"].iloc[0],
        "start": start,
        "end": end,
        "target_end_(start+4y)": target_end,
        "delta_days": delta_days,
        "strict_4y_ok": strict_ok,
        "tolerance_days": tolerance_days,
    }

four_year_checks = pd.DataFrame([check_four_year_window(g) for _, g in df.groupby("symbol")])
four_year_checks

,symbol,start,end,target_end_(start+4y),delta_days,strict_4y_ok,tolerance_days
0,Binance:AVAXUSD,2022-01-23 01:00:00,2026-01-22 01:00:00,2026-01-23 01:00:00,1460.0,True,2


In [10]:
sym = "Binance:AVAXUSD"
d = df[df["symbol"] == sym].copy()
d = d.sort_values("datetime")

expected_len = len(pd.date_range(d["datetime"].min(), d["datetime"].max(), freq="1D"))
unique_len = d["datetime"].nunique()
dup_rows = d.duplicated(subset=["datetime"]).sum()

print("Filas totales:", len(d))
print("Fechas únicas:", unique_len)
print("Duplicados (mismo datetime):", dup_rows)
print("Días esperados en rango:", expected_len)
print("Días faltantes (esperados - únicos):", expected_len - unique_len)

Filas totales: 1461
Fechas únicas: 1461
Duplicados (mismo datetime): 0
Días esperados en rango: 1461
Días faltantes (esperados - únicos): 0


In [11]:
dups = d[d.duplicated(subset=["datetime"], keep=False)].sort_values("datetime")
dups.head(20)

,datetime,symbol,open,high,low,close,volume


In [17]:
import pandas as pd
import numpy as np

SYMBOL = "Binance:AVAXUSD"

# Semana a analizar: puedes pasar un lunes exacto o cualquier fecha dentro de la semana
WEEK_DATE = "2026-01-12"   # <-- cámbialo (ej: lunes de esa semana)
WEEK_START = pd.to_datetime(WEEK_DATE) - pd.Timedelta(days=pd.to_datetime(WEEK_DATE).weekday())  # lunes
WEEK_END = WEEK_START + pd.Timedelta(days=7)  # domingo

cols = ["open","high","low","close","volume"]

d = df[df["symbol"] == SYMBOL].copy()
d["datetime"] = pd.to_datetime(d["datetime"], errors="coerce")
d = d.dropna(subset=["datetime"]).sort_values("datetime")

week = d[(d["datetime"] >= WEEK_START) & (d["datetime"] <= WEEK_END)].copy()
week["dow"] = week["datetime"].dt.day_name()
week["date"] = week["datetime"].dt.date

print("Semana:", WEEK_START.date(), "→", WEEK_END.date(), "| filas:", len(week))
week[["datetime","dow","symbol"] + cols]

Semana: 2026-01-12 → 2026-01-19 | filas: 7


,datetime,dow,symbol,open,high,low,close,volume
1450,2026-01-12 01:00:00,Monday,Binance:AVAXUSD,13.672296,14.186905,13.392508,13.493342,88520.91
1451,2026-01-13 01:00:00,Tuesday,Binance:AVAXUSD,13.498416,14.819114,13.467029,14.739847,160430.32
1452,2026-01-14 01:00:00,Wednesday,Binance:AVAXUSD,14.757407,14.939942,14.414324,14.613257,137137.12
1453,2026-01-15 01:00:00,Thursday,Binance:AVAXUSD,14.604866,14.619116,13.623548,13.817347,123202.86
1454,2026-01-16 01:00:00,Friday,Binance:AVAXUSD,13.824136,13.850380,13.195602,13.584070,98099.81
1455,2026-01-17 01:00:00,Saturday,Binance:AVAXUSD,13.573497,13.957031,13.529046,13.732979,71710.91
1456,2026-01-18 01:00:00,Sunday,Binance:AVAXUSD,13.756693,13.816375,12.944312,12.980054,77268.91
